In [40]:
from features.wiki_fns import wiki_search

wiki_search('lonely planet')

'Lonely Planet is the largest travel guide book publisher in the world. The company is owned by American billionaire Brad Kelley\'s NC2 Media, which bought it in 2013 from BBC Worldwide for US$77 million (the equivalent of 45.5 million in May 2014) after it was valued at US$250 million in 2008. Originally called "Lonely Planet Publications", the company changed its name to "Lonely Planet" in July 2009 to reflect its broad travel industry coverage and an emphasis on digital products. After the Let\'s Go travel guide series that was founded in 1960, the Lonely Planet books were the second series of travel books aimed at backpackers and other low-cost travellers. As of 2011, the company had sold 120 million books since inception and by early 2014, it had sold around 11 million units of its travel apps. As of 2014, Lonely Planet\'s largest office is located in Footscray, a suburb of Melbourne, Australia, but its Franklin, Tennessee, United States (U.S.) office is the company\'s de facto he

In [24]:
import pandas as pd
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup

In [25]:
client = MongoClient()

In [26]:
db = client.get_database("anything")

In [5]:
resp = requests.get("http://www.hillmanwonders.com/top_1000/")

In [7]:
soup = BeautifulSoup(resp.content, "lxml")

In [11]:
trs = soup.find("div", id="EdColumn").find("table").find_all("tr")

In [35]:
for tr in trs:
    tds = tr.find_all("td")
    country_td, names_td = tds[0], tds[1]
    data = {
            "country": country_td.get_text(),
            "name": names_td.get_text().strip().split("\n")
        }
    db.new_col.insert_one(data)

In [36]:
pd.DataFrame(data)

,_id,country,name
0,581bc6fdc527d56f4950379c,Covered in\nmy other lists,


In [41]:
import requests
import json
import re

def wiki_extract(select_topic):
    try:
        url = 'https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exintro=&explaintext=&titles='+select_topic+'&format=json'

        response = requests.get(url)
        jtext = response.text

        jobj = json.loads(jtext)
        s = jobj['query']['pages'][jobj['query']['pages'].keys()[0]]['extract']
        s = s.encode('ascii','ignore')

        s = re.sub('\\n', ' ', s) # remove \n
        return s
    except:
        return -1

def wiki_search(search_term):

    url = 'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch='+search_term+'&prop=extracts&exintro=&explaintext=&format=json'
    response = requests.get(url)
    jtext = response.text
    jobk = json.loads(jtext)

    i = 0
    while i<10:
        exact_title = str(jobk['query']['search'][i]['title'].encode('utf-8'))
        if 'disambiguation' in exact_title:
            i += 1
        elif 'refer' in wiki_extract(exact_title)[0:50]:
            i += 1
        else:
            break

    return wiki_extract(exact_title)


In [48]:
search_term = 'Holfburg Palace'
url = 'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch='+search_term+'&prop=extracts&exintro=&explaintext=&format=json'
response = requests.get(url)
jtext = response.text
jobk = json.loads(jtext)


In [49]:
jtext

u'{"batchcomplete":"","query":{"searchinfo":{"totalhits":0,"suggestion":"hofburg palace","suggestionsnippet":"<em>hofburg</em> palace"},"search":[]}}'

In [50]:
url

'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch=Holfburg Palace&prop=extracts&exintro=&explaintext=&format=json'